# Importar librerias

In [1]:
import janitor
import matplotlib.pyplot as plt
import missingno
import nhanes.load
import numpy as np
import pandas as pd
import scipy.stats
import seaborn as sns
import session_info
import sklearn.compose
import sklearn.impute
import sklearn.preprocessing
import statsmodels.api as sm
import statsmodels.datasets
import statsmodels.formula.api as smf

from sklearn.ensemble import RandomForestRegressor
from sklearn.experimental import enable_iterative_imputer
from sklearn.kernel_approximation import Nystroem
from sklearn.linear_model import BayesianRidge, Ridge
from sklearn.neighbors import KNeighborsRegressor
from statsmodels.graphics.mosaicplot import mosaic

import warnings

# Desactivar las advertencias de DeprecationWarning
warnings.filterwarnings("ignore")
#warnings.filterwarnings("ignore", category=DeprecationWarning)


/Users/Sebastian/Desktop/Cursos Platzi/missing_imputation/missing_env/lib/python3.9/site-packages/nhanes/load.py:5: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources
/Users/Sebastian/Desktop/Cursos Platzi/missing_imputation/missing_env/lib/python3.9/site-packages/pkg_resources/__init__.py:2868: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('zope')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/Users/Sebastian/Desktop/Cursos Platzi/missing_imputation/missing_env/lib/python3.9/site-packages/pkg_resources/__init__.py:2868: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('zope')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to

# Importar Librerias Personalizadas

In [2]:
%run pandas-missing-extension.ipynb

# Cargar Datos NHANES


In [35]:
nhanes_raw_df = (
    nhanes.load.load_NHANES_data(year="2017-2018")
    .clean_names(case_type="snake") # Para que me de los nombres separados por "_"
)


In [63]:
#print(f"El dataset de tiene {nhanes_raw_df.shape[0]} filas y {nhanes_raw_df.shape[1]} columnas. Para un totla de datos de {nhanes_raw_df.shape[0] * nhanes_raw_df.shape[1]}")

#  Porcesar los datos NHANES

In [37]:
nhanes_raw_df.select_columns("*activi*")

,vigorous_work_activity,moderate_work_activity,vigorous_recreational_activities,moderate_recreational_activities,minutes_sedentary_activity
SEQN,,,,,
93703.0,NaN,NaN,NaN,NaN,NaN
93704.0,NaN,NaN,NaN,NaN,NaN
93705.0,0.0,0.0,0.0,1.0,300.0
93706.0,0.0,0.0,0.0,1.0,240.0
93707.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
102952.0,0.0,0.0,0.0,1.0,120.0
102953.0,1.0,1.0,0.0,0.0,360.0
102954.0,0.0,0.0,0.0,1.0,600.0


In [58]:
nhanes_df = (
    nhanes_raw_df
    .select_columns(
        # Selecionamos columnas que deseamos tener del Data Frame original de NHANES
        "general_health_condition",
        "age_in_years_at_screening",
        "gender",
        "current_selfreported_height_inches",
        "current_selfreported_weight_pounds",
        "doctor_told_you_have_diabetes",
        "60_sec_pulse30_sec_pulse2",
        "total_cholesterol_mgdl",

        "vigorous_work_activity",
        "moderate_work_activity",
        "vigorous_recreational_activities",
        "moderate_recreational_activities",
        "minutes_sedentary_activity"

    )
    .rename_columns(
        # Los nombres de las columnas son complejos, podemos hacerlos mas sencillos
        # En forma de diccionario {"nombre_incial_de_columna":"new_name"}
        {
        "age_in_years_at_screening":"age",
        "current_selfreported_height_inches":"height",
        "current_selfreported_weight_pounds":"weight",
        "doctor_told_you_have_diabetes":"diebetes",
        "60_sec_pulse30_sec_pulse2":"pulse",
        "total_cholesterol_mgdl": "total_cholesterol",

        }
    )
    .replace(
        # Identificamos los valores que puedes ser nulos gracias al codigo
        # sort_values(by="name_column", ascending =True)
        {
            "height": {
                9999 : np.nan,
                7777 : np.nan
            },
            "weight": {
                9999 : np.nan,
                7777 : np.nan
            },
            "diebetes":{
                "Borderline": np.nan
            },
            "minutes_sedentary_activity":{
                9999: np.nan,
                
            }
        }
    )
    
)

In [59]:
# La columna "minutes_sedentary_activity" estaba en notaciín cientifica, era necesario pasarlo a entero
nhanes_df['minutes_sedentary_activity'] = (
    # Seleccinamos columna y que solo nos muestre valores NO nulos
    nhanes_df.loc[nhanes_df['minutes_sedentary_activity'].notna()] # Los valores NO nulos seran de la columna deseada
    .select_columns("minutes_sedentary_activity") # Seleccionamos mostrar solo columna deseada
    .astype(int) # Los convertimos a enteros
    )

In [62]:
print(f"El dataset de tiene {nhanes_df.shape[0]} filas y {nhanes_df.shape[1]} columnas. Para un totla de datos de {nhanes_df.shape[0] * nhanes_df.shape[1]}")

El dataset de tiene 8366 filas y 13 columnas. Para un totla de datos de 108758
